In [1]:
#Let's start by trying to pull some data down using Fantasy Premier League's API, which will come in JSON form
import requests as rq
import json5 as js
import json
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
fpl = rq.get("https://fantasy.premierleague.com/api/bootstrap-static/")
#This is the status of the API, if 200, all is okay, anything starting witha  4 and then it isn't okay
print(fpl.status_code)

200


In [2]:
#Download the JSON file into my github directory
fplj=fpl.json()
filepath="C:/Users/user/github/hello-world/martin-learning-python/fpl.json"
with open(filepath, 'w') as outfile:
    json.dump(fplj, outfile)

In [3]:
# Open the json file and print a list of the keys
with open(filepath) as json_data:
    d = json.load(json_data)
    print(list(d.keys()))

['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types']


In [4]:
#Elements is the the actual players, let's go look at them. 
#Using json_normalize from the pandas library we can make it into a dataframe
df = json_normalize(d['elements'])
df.head()


,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index
0,100.0,100.0,69140,0,0,-4,4,0,2,0.0,...,0,0,0,0,1,144,174.2,15.4,107.0,29.8
1,100.0,100.0,98745,0,0,-1,1,0,2,0.0,...,0,2,0,0,3,116,117.6,37.7,38.0,19.5
2,75.0,75.0,111457,0,0,-3,3,0,2,0.0,...,0,3,0,0,1,222,190.0,157.3,67.0,41.2
3,100.0,100.0,154043,0,0,-5,5,1,2,0.0,...,0,3,1,0,3,216,261.8,155.4,37.0,44.9
4,100.0,100.0,39476,0,0,-2,2,1,2,0.0,...,0,6,0,0,5,305,436.2,36.8,110.0,58.5


In [5]:
#What data is available on the players?
df.columns

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round', 'code',
       'cost_change_event', 'cost_change_event_fall', 'cost_change_start',
       'cost_change_start_fall', 'dreamteam_count', 'element_type', 'ep_next',
       'ep_this', 'event_points', 'first_name', 'form', 'id', 'in_dreamteam',
       'news', 'news_added', 'now_cost', 'photo', 'points_per_game',
       'second_name', 'selected_by_percent', 'special', 'squad_number',
       'status', 'team', 'team_code', 'total_points', 'transfers_in',
       'transfers_in_event', 'transfers_out', 'transfers_out_event',
       'value_form', 'value_season', 'web_name', 'minutes', 'goals_scored',
       'assists', 'clean_sheets', 'goals_conceded', 'own_goals',
       'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards',
       'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat',
       'ict_index'],
      dtype='object')

In [6]:
#Let's have a look at Raheem Sterling's stats for the season, first we want to index by second name
df.loc[df['second_name']=='Sterling']

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index
318,100.0,100.0,103955,0,0,-3,3,4,3,0.0,...,1,5,0,0,9,384,569.8,571.6,1334.0,247.5


In [7]:
#Not ideal though, as we're just seeing chances of playing and costs, whereas we want the interesting data! Let's select columns
start=df.columns.get_loc('minutes')
end=df.columns.get_loc('bonus')+1
also_keep=df.columns.get_loc('second_name')
also_keep2=df.columns.get_loc('total_points')
subset=df.iloc[:,np.r_[start:end,also_keep,also_keep2]]
cols=subset.columns.tolist()
cols=cols[-2:] + cols[:-2]
subset=subset[cols]
subset

,second_name,total_points,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus
0,Mustafi,26,620,0,2,2,9,0,0,0,0,0,0,1
1,Bellerín,25,623,1,0,2,10,0,0,0,2,0,0,3
2,Kolasinac,35,1103,0,2,2,19,0,0,0,3,0,0,1
3,Maitland-Niles,36,1210,0,2,3,19,0,0,0,3,1,0,3
4,Papastathopoulos,57,1696,2,0,4,25,0,0,0,6,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
623,Cundle,0,0,0,0,0,0,0,0,0,0,0,0,0
624,Buur,0,0,0,0,0,0,0,0,0,0,0,0,0
625,Giles,0,0,0,0,0,0,0,0,0,0,0,0,0
626,Cavaco Jordao,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
#Let's have another look at Raheem Sterling
subset.loc[subset['second_name']=='Sterling']

,second_name,total_points,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus
318,Sterling,118,2018,11,2,8,27,0,0,1,5,0,0,9


In [9]:
#Who are the top 10 points scorers in the game this year?
subset=subset.sort_values("total_points",ascending=False)
subset[0:10]

,second_name,total_points,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus
286,Salah,186,2243,16,6,14,15,0,0,0,1,0,0,23
319,De Bruyne,178,2148,8,18,12,20,0,0,0,2,0,0,23
287,Mané,175,2077,14,9,9,18,0,0,0,2,0,0,22
254,Vardy,167,2222,19,6,8,25,0,0,1,2,0,0,27
277,Alexander-Arnold,166,2549,2,14,11,21,0,0,0,4,0,0,18
10,Aubameyang,152,2303,17,3,7,33,0,0,0,3,1,0,29
605,Jiménez,147,2458,13,6,9,31,0,0,0,3,0,0,23
278,van Dijk,141,2610,4,1,12,21,0,0,0,1,0,0,12
472,Ings,140,1993,15,1,3,36,0,0,0,2,0,0,28
276,Robertson,137,2443,1,7,11,20,0,0,0,1,0,0,15
